# Boston Property and Crime Cleanup
This notebook rebuilds the preprocessing pipeline for Boston property assessment data (2020-2025) and Boston Police Department incident data. It standardizes schemas, engineers features, and exports cleaned tables for downstream modeling and visualization.

## Goals
- consolidate yearly property assessment extracts into a normalized schema
- engineer housing value features with light imputation and winsorization
- clean Boston incident records for spatial and temporal aggregation
- export tidy csv/parquet outputs under `processed/` for future notebooks and apps

In [ ]:

from pathlib import Path
import pandas as pd
import numpy as np
import re
import warnings
from IPython.display import display

warnings.filterwarnings("ignore")

pd.set_option("display.max_columns", 60)
pd.set_option("display.width", 120)

NB_DIR = Path.cwd().resolve()
ROOT = NB_DIR.parents[1]
RAW_PROPERTY_FILES = [
    ROOT / "backend"/ "data" / "data2020-full.csv",
    ROOT / "backend"/ "data" / "data2021-full.csv",
    ROOT / "backend"/ "data" / "fy2022pa-4.csv",
    ROOT / "backend"/ "data" / "fy2023-property-assessment-data.csv",
    ROOT / "backend"/ "data" / "fy2024-property-assessment-data_1_5_2024.csv",
    ROOT / "backend"/ "data" / "fy2025-property-assessment-data_12_30_2024.csv",
]
CRIME_FILE = ROOT / "backend"/ "data" / "CrimeData.csv"
OUT_DIR = NB_DIR / "processed"
OUT_DIR.mkdir(parents=True, exist_ok=True)

print("Notebook directory:", NB_DIR)
print("Project root:", ROOT)
print("Processed output dir:", OUT_DIR)
print("Property source files:")
for path in RAW_PROPERTY_FILES:
    print(" -", path.name, "FOUND" if path.exists() else "MISSING")
print("Crime file:", CRIME_FILE.name, "FOUND" if CRIME_FILE.exists() else "MISSING")


Notebook directory: /Users/jeet/github/Boston-Area-Price-and-Crime-Prediction/backend/notebooks
Project root: /Users/jeet/github/Boston-Area-Price-and-Crime-Prediction
Processed output dir: /Users/jeet/github/Boston-Area-Price-and-Crime-Prediction/backend/notebooks/processed
Property source files:
 - data2020-full.csv FOUND
 - data2021-full.csv FOUND
 - fy2022pa-4.csv FOUND
 - fy2023-property-assessment-data.csv FOUND
 - fy2024-property-assessment-data_1_5_2024.csv FOUND
 - fy2025-property-assessment-data_12_30_2024.csv FOUND
Crime file: CrimeData.csv FOUND


## Utilities
Helper functions shared across the property and crime pipelines.

In [2]:

from typing import Dict, List, Tuple


def safe_read_csv(path: Path, **kwargs) -> pd.DataFrame:
    """Read a csv defensively, returning an empty frame if the file is missing or malformed."""
    if not path.exists():
        print(f"WARN: missing file {path}")
        return pd.DataFrame()
    try:
        return pd.read_csv(path, low_memory=False, encoding_errors="ignore", **kwargs)
    except Exception as exc:  # noqa: BLE001
        print(f"ERROR reading {path.name}: {exc}")
        return pd.DataFrame()


def detect_year_from_filename(path: Path) -> float:
    match = re.search(r"(20\d{2})", path.name)
    return int(match.group(1)) if match else np.nan


def to_numeric_clean(series: pd.Series) -> pd.Series:
    if series is None or series.empty:
        return pd.Series(dtype="float64")
    cleaned = series.astype(str).str.replace(r"[^0-9\.-]", "", regex=True)
    return pd.to_numeric(cleaned, errors="coerce")


FIELD_PATTERNS: Dict[str, List[str]] = {
    "parcel_id": [r"\bPID\b", r"PARCEL", r"GIS_ID"],
    "street_num": [r"ST[_ ]?NUM", r"STREET_NUM"],
    "street_name": [r"ST[_ ]?NAME(?!_SUF)", r"STREET_?NAME"],
    "street_suffix": [r"ST[_ ]?NAME[_ ]?SUF", r"STREET[_ ]?SUF"],
    "unit": [r"\bUNIT\b"],
    "city": [r"\bCITY\b"],
    "zip": [r"ZIP.?CODE", r"\bZIP\b"],
    "res_type": [r"RES[_ ]?TYPE", r"\bRES\b", r"\bLU\b", r"PTYPE", r"LUC", r"USE"],
    "building_style": [r"STYLE", r"BLDG[_ ]?STYLE", r"LU_DESC"],
    "land_sf": [r"LAND[_ ]?SF", r"LOT[_ ]?SIZE", r"LAND AREA"],
    "living_area": [r"LIVING[_ ]?AREA", r"GROSS[_ ]?AREA", r"TOTAL[_ ]?LIVING"],
    "year_built": [r"YR[_ ]?BUILT", r"YEAR[_ ]?BUILT"],
    "total_value": [r"TOTAL[_ ]?VALUE", r"AV[_ ]?TOTAL", r"ASSESS.*TOTAL"],
    "latitude": [r"\bLAT(ITUDE)?\b"],
    "longitude": [r"\bLON(GITUDE)?\b"],
}


def find_column(columns: List[str], patterns: List[str]) -> str | None:
    for pattern in patterns:
        for col in columns:
            if re.search(pattern, col, flags=re.IGNORECASE):
                return col
    return None


def clean_string_series(series: pd.Series | None) -> pd.Series:
    if series is None:
        return pd.Series(dtype="object")

    cleaned = series.astype("object")
    if cleaned.empty:
        return pd.Series([None] * len(cleaned), index=cleaned.index, dtype="object")

    cleaned = cleaned.where(~pd.isna(cleaned), None)

    def _strip(value):
        if value is None:
            return None
        if isinstance(value, str):
            return value.strip()
        return str(value).strip()

    cleaned = cleaned.apply(_strip)
    cleaned = cleaned.replace({"": None, "nan": None, "NaN": None, "NONE": None, "None": None})
    return cleaned.astype("object")


def compose_address(street_num, street_name, unit, city, zip_code) -> str:
    parts = []
    if street_num and str(street_num).strip().lower() not in {"nan", "none"}:
        parts.append(str(street_num).strip())
    if street_name and str(street_name).strip().lower() not in {"nan", "none"}:
        parts.append(str(street_name).strip())
    base = " ".join(parts).strip()
    if unit and str(unit).strip().lower() not in {"nan", "none"}:
        base = f"{base}, Unit {str(unit).strip()}" if base else f"Unit {str(unit).strip()}"
    if city and str(city).strip().lower() not in {"nan", "none"}:
        formatted_city = str(city).strip().title()
        base = f"{base}, {formatted_city}" if base else formatted_city
    if zip_code and str(zip_code).strip().isdigit():
        base = f"{base}, {str(zip_code).strip()}" if base else str(zip_code).strip()
    return base.strip().strip(",")


def winsorize_grouped(df: pd.DataFrame, col: str, group: str, lower: float = 0.01, upper: float = 0.99) -> pd.DataFrame:
    if col not in df.columns or group not in df.columns:
        return df

    def _clip(s: pd.Series) -> pd.Series:
        if s.dropna().empty:
            return s
        lower_q = s.quantile(lower)
        upper_q = s.quantile(upper)
        return s.clip(lower_q, upper_q)

    df[col] = df.groupby(group)[col].transform(_clip)
    return df


def hierarchical_impute(df: pd.DataFrame, cols: List[str]) -> pd.DataFrame:
    for col in cols:
        if col not in df.columns:
            continue
        if {"zip", "res_type"}.issubset(df.columns):
            df[col] = df[col].fillna(df.groupby(["zip", "res_type"])[col].transform("median"))
        if "zip" in df.columns:
            df[col] = df[col].fillna(df.groupby("zip")[col].transform("median"))
        df[col] = df[col].fillna(df[col].median())
    return df


def safe_to_parquet(df: pd.DataFrame, path: Path) -> bool:
    """Write parquet if engine available; return True when successful."""
    try:
        df.to_parquet(path, index=False)
        return True
    except ImportError as err:
        print(f"WARN: {err}. Skipping parquet write for {path.name}.")
    except ValueError as err:
        print(f"WARN: Failed to write {path.name}: {err}")
    return False


def top_mode(series: pd.Series):
    counts = series.dropna().value_counts()
    return counts.index[0] if not counts.empty else np.nan


def make_daypart(hour) -> str | float:
    if pd.isna(hour):
        return np.nan
    try:
        hour_int = int(hour)
    except (TypeError, ValueError):  # noqa: PERF203
        return np.nan
    if 5 <= hour_int < 12:
        return "morning"
    if 12 <= hour_int < 17:
        return "afternoon"
    if 17 <= hour_int < 21:
        return "evening"
    return "overnight"


BOSTON_BOUNDS = {"lat_min": 42.0, "lat_max": 42.6, "lon_min": -71.2, "lon_max": -70.9}


## Property pipeline
Standardize yearly assessment files, engineer housing metrics, and export clean tables.

In [3]:

PROPERTY_CPI_FACTORS = {2020: 1.20, 2021: 1.12, 2022: 1.08, 2023: 1.04, 2024: 1.02, 2025: 1.00}
CURRENT_YEAR = 2025


def canonicalize_property_file(path: Path) -> Tuple[pd.DataFrame, Dict[str, str]]:
    raw = safe_read_csv(path)
    diagnostics: Dict[str, str] = {"source": path.name, "rows": len(raw)}
    if raw.empty:
        diagnostics["missing_columns"] = list(FIELD_PATTERNS.keys())
        return pd.DataFrame(), diagnostics

    columns = list(raw.columns)
    mapping = {field: find_column(columns, patterns) for field, patterns in FIELD_PATTERNS.items()}
    diagnostics["mapping"] = mapping
    diagnostics["missing_columns"] = [field for field, col in mapping.items() if col is None]

    out = pd.DataFrame(index=raw.index)
    out["source_file"] = path.name
    out["source_year"] = detect_year_from_filename(path)

    def take(field: str) -> pd.Series:
        col = mapping.get(field)
        if col is None or col not in raw.columns:
            return pd.Series(np.nan, index=raw.index)
        return raw[col]

    street_num = clean_string_series(take("street_num")).str.replace(r"\.0$", "", regex=True)
    street_name = clean_string_series(take("street_name"))
    suffix_col = mapping.get("street_suffix")
    if suffix_col:
        suffix = clean_string_series(raw[suffix_col])
        street_name = (street_name.fillna("") + " " + suffix.fillna("")).str.replace(r"\s+", " ", regex=True).str.strip()
        street_name = street_name.replace({"": np.nan})
    unit = clean_string_series(take("unit"))
    city = clean_string_series(take("city")).str.title()
    zip_code = take("zip").astype(str).str.extract(r"(\d{5})", expand=False)

    out["parcel_id"] = clean_string_series(take("parcel_id")).str.replace(r"\.0$", "", regex=True).str.upper()
    out["street_num"] = street_num
    out["street_name"] = street_name
    out["unit"] = unit
    out["city"] = city
    out["zip"] = zip_code
    out["res_type"] = clean_string_series(take("res_type"))
    out["building_style"] = clean_string_series(take("building_style"))
    out["land_sf"] = to_numeric_clean(take("land_sf"))
    out["living_area"] = to_numeric_clean(take("living_area"))
    out["year_built"] = to_numeric_clean(take("year_built"))
    out["total_value"] = to_numeric_clean(take("total_value"))
    out["latitude"] = pd.to_numeric(take("latitude"), errors="coerce")
    out["longitude"] = pd.to_numeric(take("longitude"), errors="coerce")
    out["year"] = out["source_year"]
    out["addr_std"] = [
        compose_address(sn, st, un, ct, zp)
        for sn, st, un, ct, zp in zip(street_num, street_name, unit, city, zip_code)
    ]
    out["addr_std"] = pd.Series(out["addr_std"], index=out.index).str.replace(r"\s+", " ", regex=True).str.strip()
    out.loc[out["addr_std"].isin(["", "nan", "None", "NONE"]), "addr_std"] = np.nan

    return out, diagnostics


PROPERTY_KEEP_COLS = [
    "parcel_id",
    "addr_std",
    "street_num",
    "street_name",
    "unit",
    "city",
    "zip",
    "res_type",
    "building_style",
    "land_sf",
    "living_area",
    "year_built",
    "total_value",
    "total_value_real_2025",
    "price_per_sqft",
    "lot_coverage",
    "property_age",
    "latitude",
    "longitude",
    "year",
    "source_file",
]


In [4]:

property_frames: List[pd.DataFrame] = []
property_diagnostics: List[Dict] = []

for path in RAW_PROPERTY_FILES:
    frame, diag = canonicalize_property_file(path)
    property_diagnostics.append(diag)
    if frame.empty:
        print(f"{path.name}: skipped (empty)")
        continue
    missing = diag.get("missing_columns", [])
    missing_msg = "" if not missing else f" | missing: {', '.join(missing)}"
    print(f"{path.name}: {frame.shape}{missing_msg}")
    property_frames.append(frame)

if property_frames:
    property_all = pd.concat(property_frames, ignore_index=True)
else:
    property_all = pd.DataFrame()

print("Combined property rows:", len(property_all))

if not property_all.empty:
    property_all["parcel_id"] = property_all["parcel_id"].str.replace(r"[^A-Z0-9]", "", regex=True)
    property_all["parcel_id"] = property_all["parcel_id"].replace({"": np.nan})
    property_all = property_all.dropna(subset=["parcel_id"]).copy()

    property_all["zip"] = property_all["zip"].where(property_all["zip"].str.fullmatch(r"\d{5}", na=False), np.nan)
    property_all["addr_std"] = property_all["addr_std"].str.replace(r"\s+", " ", regex=True).str.strip()

    property_all["year"] = property_all["year"].fillna(property_all["source_year"])
    property_all["year"] = pd.to_numeric(property_all["year"], errors="coerce").astype("Int64")

    property_all = hierarchical_impute(property_all, ["living_area", "land_sf", "year_built"])

    property_all["price_per_sqft"] = property_all["total_value"] / property_all["living_area"].replace({0: np.nan})
    property_all.loc[~np.isfinite(property_all["price_per_sqft"]), "price_per_sqft"] = np.nan

    property_all["lot_coverage"] = property_all["living_area"] / property_all["land_sf"].replace({0: np.nan})
    property_all.loc[~np.isfinite(property_all["lot_coverage"]), "lot_coverage"] = np.nan

    property_all["total_value_real_2025"] = property_all["total_value"] * property_all["year"].map(PROPERTY_CPI_FACTORS).fillna(1.0)
    property_all["property_age"] = CURRENT_YEAR - property_all["year_built"]

    for col in ["price_per_sqft", "total_value_real_2025"]:
        property_all = winsorize_grouped(property_all, col, "year")

    property_all = property_all[
        (property_all["living_area"].between(150, 10000) | property_all["living_area"].isna())
    ]
    property_all = property_all[
        (property_all["land_sf"].between(300, 200000) | property_all["land_sf"].isna())
    ]
    property_all = property_all[
        (property_all["total_value"].between(10000, 10000000) | property_all["total_value"].isna())
    ]

    inside_bounds = (
        property_all["latitude"].between(BOSTON_BOUNDS["lat_min"], BOSTON_BOUNDS["lat_max"], inclusive="both")
        & property_all["longitude"].between(BOSTON_BOUNDS["lon_min"], BOSTON_BOUNDS["lon_max"], inclusive="both")
    )
    coord_missing = property_all["latitude"].isna() & property_all["longitude"].isna()
    property_all = property_all[inside_bounds | coord_missing]

    property_clean = property_all[PROPERTY_KEEP_COLS].copy()
else:
    property_clean = property_all.copy()

print("property_clean shape:", property_clean.shape)


data2020-full.csv: (175052, 18) | missing: unit, city, latitude, longitude
data2021-full.csv: (177091, 18) | missing: street_suffix, unit, latitude, longitude
fy2022pa-4.csv: (178598, 18) | missing: street_suffix, unit, latitude, longitude
fy2023-property-assessment-data.csv: (180627, 18) | missing: street_suffix, unit, latitude, longitude
fy2024-property-assessment-data_1_5_2024.csv: (182242, 18) | missing: street_suffix, unit, latitude, longitude
fy2025-property-assessment-data_12_30_2024.csv: (183445, 18) | missing: street_suffix, unit, latitude, longitude
Combined property rows: 1077055
property_clean shape: (872908, 21)


In [5]:
property_outputs = []
if not property_clean.empty:
    property_clean_path_csv = OUT_DIR / "property_assessments_clean.csv"
    property_clean_path_parquet = OUT_DIR / "property_assessments_clean.parquet"
    property_clean.to_csv(property_clean_path_csv, index=False)
    property_outputs.append(property_clean_path_csv)
    if safe_to_parquet(property_clean, property_clean_path_parquet):
        property_outputs.append(property_clean_path_parquet)

    property_by_zip_year = (
        property_clean.dropna(subset=["zip", "year"])
        .groupby(["zip", "year"], as_index=False)
        .agg(
            parcel_count=("parcel_id", "nunique"),
            median_total_value=("total_value", "median"),
            median_total_value_real_2025=("total_value_real_2025", "median"),
            median_price_per_sqft=("price_per_sqft", "median"),
            median_living_area=("living_area", "median"),
        )
        .sort_values(["year", "zip"])
    )
    property_by_zip_year_path = OUT_DIR / "property_by_zip_year.csv"
    property_by_zip_year.to_csv(property_by_zip_year_path, index=False)
    property_outputs.append(property_by_zip_year_path)

    property_by_res_type_year = (
        property_clean.dropna(subset=["res_type", "year"])
        .groupby(["res_type", "year"], as_index=False)
        .agg(
            parcel_count=("parcel_id", "nunique"),
            median_total_value=("total_value", "median"),
            median_price_per_sqft=("price_per_sqft", "median"),
            median_living_area=("living_area", "median"),
        )
        .sort_values(["res_type", "year"])
    )
    property_by_res_type_year_path = OUT_DIR / "property_by_res_type_year.csv"
    property_by_res_type_year.to_csv(property_by_res_type_year_path, index=False)
    property_outputs.append(property_by_res_type_year_path)

    print("Saved property outputs:")
    for path in property_outputs:
        print(" -", path)
else:
    print("No property rows available; skipped property exports.")


Saved property outputs:
 - /Users/jeet/github/Boston-Area-Price-and-Crime-Prediction/backend/notebooks/processed/property_assessments_clean.csv
 - /Users/jeet/github/Boston-Area-Price-and-Crime-Prediction/backend/notebooks/processed/property_assessments_clean.parquet
 - /Users/jeet/github/Boston-Area-Price-and-Crime-Prediction/backend/notebooks/processed/property_by_zip_year.csv
 - /Users/jeet/github/Boston-Area-Price-and-Crime-Prediction/backend/notebooks/processed/property_by_res_type_year.csv


In [6]:

if not property_clean.empty:
    display_cols = ["parcel_id", "addr_std", "zip", "total_value", "price_per_sqft", "year"]
    sample_preview = property_clean[display_cols].sample(n=min(5, len(property_clean)), random_state=42)
    print("Sample property rows:")
    display(sample_preview)

    print("Median property metrics by year:")
    medians_by_year = property_clean.groupby("year")[
        ["total_value", "total_value_real_2025", "price_per_sqft", "living_area"]
    ].median().rename(columns={
        "total_value": "median_total_value",
        "total_value_real_2025": "median_value_real_2025",
        "price_per_sqft": "median_price_per_sqft",
        "living_area": "median_living_area",
    })
    display(medians_by_year)
else:
    print("Property dataset is empty; nothing to profile.")


Sample property rows:


,parcel_id,addr_std,zip,total_value,price_per_sqft,year
1016122,1702956000,"116 MILTON AV, Dorchester",NaN,702000.0,317.073171,2025
1052242,2006953002,"52 WESTOVER ST, West Roxbury",NaN,486000.0,392.251816,2025
203348,304850788,"151 TREMONT ST, Boston",NaN,1097100.0,979.553571,2021
514889,2100772020,"1161 COMMONWEALTH AV, Allston",NaN,337500.0,519.230769,2022
418382,604081000,"160 M ST, South Boston",NaN,759900.0,538.936170,2022


Median property metrics by year:


,median_total_value,median_value_real_2025,median_price_per_sqft,median_living_area
year,,,,
2020,572600.0,687120.0,263.174857,2243.0
2021,588700.0,659344.0,393.061532,1440.0
2022,609800.0,658584.0,409.444444,1440.0
2023,632600.0,657904.0,427.455357,1440.0
2024,657650.0,670803.0,444.526642,1440.0
2025,686200.0,686200.0,465.198413,1440.0


## Crime pipeline
Clean Boston incident records, engineer convenience features, and export aggregated views.

In [7]:

CRIME_COLUMN_RENAMES = {
    "INCIDENT_NUMBER": "incident_number",
    "OFFENSE_CODE": "offense_code",
    "OFFENSE_CODE_GROUP": "offense_code_group",
    "OFFENSE_DESCRIPTION": "offense_description",
    "DISTRICT": "district",
    "REPORTING_AREA": "reporting_area",
    "SHOOTING": "shooting",
    "OCCURRED_ON_DATE": "occurred_on_date",
    "YEAR": "reported_year",
    "MONTH": "reported_month",
    "DAY_OF_WEEK": "day_of_week",
    "HOUR": "hour",
    "UCR_PART": "ucr_part",
    "STREET": "street",
    "Lat": "latitude",
    "Long": "longitude",
}


def clean_crime_data(path: Path) -> Tuple[pd.DataFrame, Dict]:
    raw = safe_read_csv(path)
    diagnostics = {"source": path.name, "rows": len(raw)}
    if raw.empty:
        return pd.DataFrame(), diagnostics

    df = raw.rename(columns={col: CRIME_COLUMN_RENAMES.get(col, col.lower()) for col in raw.columns})

    df["occurred_on_dt"] = pd.to_datetime(df.get("occurred_on_date"), errors="coerce")
    df["year"] = df["occurred_on_dt"].dt.year
    df["month"] = df["occurred_on_dt"].dt.month
    df["hour"] = pd.to_numeric(df.get("hour"), errors="coerce")
    df = df[df["year"].between(2020, 2025, inclusive="both")]

    df["incident_number"] = df.get("incident_number", df.get("INCIDENT_NUMBER"))
    df["incident_number"] = df["incident_number"].astype(str).str.strip()

    df["district"] = df.get("district", df.get("DISTRICT")).fillna("UNK").astype(str).str.upper()
    df["reporting_area"] = (
        df.get("reporting_area", df.get("REPORTING_AREA"))
        .astype(str)
        .str.replace(r"\.0$", "", regex=True)
        .str.zfill(3)
    )

    df["latitude"] = pd.to_numeric(df.get("latitude"), errors="coerce")
    df["longitude"] = pd.to_numeric(df.get("longitude"), errors="coerce")
    inside_bounds = (
        df["latitude"].between(BOSTON_BOUNDS["lat_min"], BOSTON_BOUNDS["lat_max"], inclusive="both")
        & df["longitude"].between(BOSTON_BOUNDS["lon_min"], BOSTON_BOUNDS["lon_max"], inclusive="both")
    )
    coord_missing = df["latitude"].isna() & df["longitude"].isna()
    df = df[inside_bounds | coord_missing]

    df["shooting"] = df.get("shooting", df.get("SHOOTING"))
    df["is_shooting"] = df["shooting"].astype(str).str.upper().isin(["Y", "YES", "TRUE", "1"])

    df["incident_daypart"] = df["hour"].apply(make_daypart)
    df["weekday"] = df["occurred_on_dt"].dt.day_name()

    df["offense_code_group"] = df.get("offense_code_group", df.get("OFFENSE_CODE_GROUP"))
    df["offense_description"] = df.get("offense_description", df.get("OFFENSE_DESCRIPTION"))

    df = df.sort_values("occurred_on_dt").drop_duplicates(subset=["incident_number"], keep="last")

    keep_cols = [
        "incident_number",
        "offense_code",
        "offense_code_group",
        "offense_description",
        "district",
        "reporting_area",
        "is_shooting",
        "occurred_on_dt",
        "year",
        "month",
        "hour",
        "incident_daypart",
        "weekday",
        "latitude",
        "longitude",
        "street",
        "ucr_part",
    ]
    missing_keep = [col for col in keep_cols if col not in df.columns]
    if missing_keep:
        diagnostics["missing_columns"] = missing_keep

    crime_clean = df.reindex(columns=keep_cols)
    return crime_clean, diagnostics


In [8]:
crime_clean, crime_diag = clean_crime_data(CRIME_FILE)
print("crime_clean shape:", crime_clean.shape)
crime_outputs = []
if crime_clean.empty:
    print("No crime data available; skipping crime exports.")
else:
    crime_clean_path_csv = OUT_DIR / "crime_incidents_clean.csv"
    crime_clean_path_parquet = OUT_DIR / "crime_incidents_clean.parquet"
    crime_clean.to_csv(crime_clean_path_csv, index=False)
    crime_outputs.append(crime_clean_path_csv)

    if safe_to_parquet(crime_clean, crime_clean_path_parquet):
        crime_outputs.append(crime_clean_path_parquet)

    crime_by_district_month = (
        crime_clean.groupby(["district", "year", "month"], as_index=False)
        .agg(
            incident_count=("incident_number", "count"),
            shooting_incidents=("is_shooting", "sum"),
            predominant_offense=("offense_code_group", top_mode),
        )
        .sort_values(["year", "month", "district"])
    )
    crime_by_district_month_path = OUT_DIR / "crime_by_district_month.csv"
    crime_by_district_month.to_csv(crime_by_district_month_path, index=False)
    crime_outputs.append(crime_by_district_month_path)

    crime_by_reporting_area = (
        crime_clean.groupby(["reporting_area", "year", "month"], as_index=False)
        .agg(
            incident_count=("incident_number", "count"),
            shooting_incidents=("is_shooting", "sum"),
            predominant_offense=("offense_code_group", top_mode),
        )
        .sort_values(["year", "month", "reporting_area"])
    )
    crime_by_reporting_area_path = OUT_DIR / "crime_by_reporting_area_month.csv"
    crime_by_reporting_area.to_csv(crime_by_reporting_area_path, index=False)
    crime_outputs.append(crime_by_reporting_area_path)

    crime_by_year = (
        crime_clean.groupby("year", as_index=False)
        .agg(
            incident_count=("incident_number", "count"),
            shooting_incidents=("is_shooting", "sum"),
        )
        .sort_values("year")
    )
    crime_by_year_path = OUT_DIR / "crime_by_year.csv"
    crime_by_year.to_csv(crime_by_year_path, index=False)
    crime_outputs.append(crime_by_year_path)

    print("Saved crime outputs:")
    for path in crime_outputs:
        print(" -", path)

print("Crime diagnostics:", crime_diag)


crime_clean shape: (214626, 17)
Saved crime outputs:
 - /Users/jeet/github/Boston-Area-Price-and-Crime-Prediction/backend/notebooks/processed/crime_incidents_clean.csv
 - /Users/jeet/github/Boston-Area-Price-and-Crime-Prediction/backend/notebooks/processed/crime_incidents_clean.parquet
 - /Users/jeet/github/Boston-Area-Price-and-Crime-Prediction/backend/notebooks/processed/crime_by_district_month.csv
 - /Users/jeet/github/Boston-Area-Price-and-Crime-Prediction/backend/notebooks/processed/crime_by_reporting_area_month.csv
 - /Users/jeet/github/Boston-Area-Price-and-Crime-Prediction/backend/notebooks/processed/crime_by_year.csv
Crime diagnostics: {'source': 'CrimeData.csv', 'rows': 214628}


In [9]:

if not crime_clean.empty:
    preview_cols = [
        "incident_number",
        "district",
        "occurred_on_dt",
        "incident_daypart",
        "offense_code_group",
        "is_shooting",
    ]
    preview = crime_clean[preview_cols].sort_values("occurred_on_dt", ascending=False).head(5)
    print("Recent crime incidents:")
    display(preview)

    print("Monthly incident counts (last 12 months if available):")
    recent_months = (
        crime_clean.sort_values("occurred_on_dt")
        .groupby(["year", "month"])
        .agg(
            incident_count=("incident_number", "count"),
            shooting_incidents=("is_shooting", "sum"),
        )
        .tail(12)
    )
    display(recent_months)
else:
    print("Crime dataset is empty; nothing to profile.")


Recent crime incidents:


,incident_number,district,occurred_on_dt,incident_daypart,offense_code_group,is_shooting
14099,252078778,E18,2025-09-20 04:25:00+00:00,overnight,NaN,False
85812,252078772,D4,2025-09-20 02:52:00+00:00,overnight,NaN,False
85865,252078763,D4,2025-09-20 02:38:00+00:00,overnight,NaN,False
85848,252078755,B3,2025-09-20 02:08:00+00:00,overnight,NaN,False
78868,252078754,D4,2025-09-20 02:08:00+00:00,overnight,NaN,False


Monthly incident counts (last 12 months if available):


incident_count  shooting_incidents
year month                                    
2024 10               6826                  37
     11               6523                  42
     12               6761                  32
2025 1                6206                  32
     2                5619                  31
     3                6704                  42
     4                6424                  51
     5                7142                  45
     6                6702                  53
     7                7089                  60
     8                7263                  59
     9                4429                  29

## Output summary
Quick reference of files generated by this notebook.

In [10]:
all_outputs = []
if 'property_outputs' in globals():
    all_outputs.extend(property_outputs)
if 'crime_outputs' in globals():
    all_outputs.extend(crime_outputs)

if all_outputs:
    print("Artifacts written to:")
    for path in all_outputs:
        print(" -", path)
else:
    print("No artifacts were written (check diagnostics above).")


Artifacts written to:
 - /Users/jeet/github/Boston-Area-Price-and-Crime-Prediction/backend/notebooks/processed/property_assessments_clean.csv
 - /Users/jeet/github/Boston-Area-Price-and-Crime-Prediction/backend/notebooks/processed/property_assessments_clean.parquet
 - /Users/jeet/github/Boston-Area-Price-and-Crime-Prediction/backend/notebooks/processed/property_by_zip_year.csv
 - /Users/jeet/github/Boston-Area-Price-and-Crime-Prediction/backend/notebooks/processed/property_by_res_type_year.csv
 - /Users/jeet/github/Boston-Area-Price-and-Crime-Prediction/backend/notebooks/processed/crime_incidents_clean.csv
 - /Users/jeet/github/Boston-Area-Price-and-Crime-Prediction/backend/notebooks/processed/crime_incidents_clean.parquet
 - /Users/jeet/github/Boston-Area-Price-and-Crime-Prediction/backend/notebooks/processed/crime_by_district_month.csv
 - /Users/jeet/github/Boston-Area-Price-and-Crime-Prediction/backend/notebooks/processed/crime_by_reporting_area_month.csv
 - /Users/jeet/github/Bosto